# Semantic Segmentation Quiz

### Maintainer : Wonkwang Lee (wonkwang.lee@kaist.ac.kr)

In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

sess = tf.InteractiveSession()
x = tf.placeholder(tf.float32, shape=(1, 224, 224, 3))
label = tf.broadcast_to(tf.expand_dims(tf.expand_dims(tf.one_hot(1, 21), -1), -1), (1, 21, 224, 224))

# Quiz 1: Building toy FCN blocks

![FCN](http://deeplearning.net/tutorial/_images/cat_segmentation.png)


## Instruction:
Build toy FCN blocks using `models.Sequential()`, `layers.Conv2D`, `layers.Maxpooling2D`, `layer.UpSampling2D`. <br>
Please refer to the printed output for `fcn.summary()`

#### hint1 : https://www.tensorflow.org/beta/tutorials/images/intro_to_cnns
#### hint2 : https://keras.io/layers/convolutional/#conv2d
#### hint3 : https://keras.io/layers/pooling/#maxpooling2d
#### hint4 : https://keras.io/layers/convolutional/#upsampling2d

In [2]:
def FCN():
    model = models.Sequential()
    model.add(layers.Conv2D(96, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
    ''' implement remaining modules here '''
    #model.add(...)
    return model

In [3]:
fcn = FCN()
fcn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 96)      2688      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 96)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 256)     221440    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 384)       885120    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 56, 56, 384)       1327488   
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 56, 56, 256)       884992    
__________

In [4]:
print(x.shape)

(1, 224, 224, 3)


In [5]:
prediction = fcn(x)
prediction

<tf.Tensor 'sequential/up_sampling2d/ResizeNearestNeighbor:0' shape=(1, 224, 224, 21) dtype=float32>

---

# Quiz 2: Preprocess data to calculate cross-entropy loss


#### hint1 : https://www.tensorflow.org/api_docs/python/tf/losses/softmax_cross_entropy
#### hint2 : https://www.tensorflow.org/api_docs/python/tf/transpose

In [6]:
def preprocess_label(label):
    ''' write down proper pre-processing operation here'''
    #label = ...
    return label

def calculate_loss(prediction, label):
    loss = tf.losses.softmax_cross_entropy(label, prediction)
    return loss

In [7]:
print('prediction shape :', prediction.shape)
print('label shape :', label.shape)

prediction shape : (1, 224, 224, 21)
label shape : (1, 21, 224, 224)


In [8]:
label = preprocess_label(label)
# print('label shape :', label.shape)

In [9]:
loss = calculate_loss(prediction, label)
loss

<tf.Tensor 'softmax_cross_entropy_loss/value:0' shape=() dtype=float32>

---

# Quiz 3: Building toy DeconvNet

![deconvnet](https://miro.medium.com/max/1400/1*LW8Anre45o9nfamxIVTY8Q.png)

## Instruction:
Build toy DeconvNet blocks using `models.Sequential()`, `layers.Conv2D`, `layers.Maxpooling2D`, `layer.Conv2DTranspose`. <br>
Please refer to the printed output for `deconvnet.summary()` <br>
<br>
Note: we use `layer.Conv2DTranspose` for upsampling feature maps instead of un-pooling operation

In [10]:
def DeconvNet():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
    ''' implement remaining modules here '''
    #model.add(...)
    return model

In [11]:
deconvnet = DeconvNet()
deconvnet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 112, 112, 64)      18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 56, 56, 128)       73856     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 28, 28, 128)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 28, 28, 256)       295168    
__________

In [12]:
prediction = deconvnet(x)
prediction

<tf.Tensor 'sequential_1/conv2d_transpose_4/Relu:0' shape=(1, 224, 224, 21) dtype=float32>

---

# Quiz 4 : Feature Pyramid

<img src="toy_fpn.png" width="600">

## Instruction
Implement `upsample_and_merge` function

#### hint1 : https://www.tensorflow.org/api_docs/python/tf/concat

In [13]:
conv_block1 = models.Sequential()
conv_block1.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
conv_block1.add(layers.MaxPool2D((2,2)))

conv_block2 = models.Sequential()
conv_block2.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
conv_block2.add(layers.MaxPool2D((2,2)))

In [14]:
out1 = conv_block1(x)
out1

<tf.Tensor 'sequential_2/max_pooling2d_8/MaxPool:0' shape=(1, 112, 112, 32) dtype=float32>

In [15]:
out2 = conv_block2(out1)
out2

<tf.Tensor 'sequential_3/MaxPool:0' shape=(1, 56, 56, 64) dtype=float32>

In [16]:
def upsample_and_merge(small, large):
    '''write down codes here'''
    # merged = ...
    return merged

In [17]:
upsample_and_merge(out2, out1)

<tf.Tensor 'concat:0' shape=(1, 112, 112, 96) dtype=float32>

---

# Quiz 5 : Building blocks with atrous convolution

![fcn_atrous](fcn_atrous.png)
#### hint1 : https://keras.io/layers/convolutional/#conv2d

In [18]:
def Conv_Atrous():
    model = models.Sequential()
    ''' implement remaining modules here. note that the last four modules should be atrous convs'''
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
    #model.add(...)
    #...
    # add atrous convolution with rate 2
    #model.add(...)
    # add atrous convolution with rate 4
    #model.add(...)
    # add atrous convolution with rate 8
    #model.add(...)
    # add atrous convolution with rate 16
    #model.add(...)
    return model

In [19]:
conv_atrous = Conv_Atrous()
conv_atrous.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 112, 112, 64)      18496     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 56, 56, 128)       73856     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 28, 28, 128)       0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 28, 28, 256)       295168    
__________

---

# Quiz 6 : Building toy DeepLab

![aspp](aspp.png)

## Instruction
Complete ``DeepLab_Front`, conv_with_dilation_rate2`, `conv_with_dilation_rate4`, `conv_with_dilation_rate8`, `conv_with_dilation_rate16`, and `aspp`, where `aspp` is simply computed by concatenating all the `feature_map_rateX`

In [20]:
def DeepLab_Front():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
    ''' implement remaining modules here '''
    #model.add(...)
    return model

In [21]:
front = DeepLab_Front()
front.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 112, 112, 64)      18496     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 56, 56, 128)       73856     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 28, 28, 128)       0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 28, 28, 256)       295168    
__________

In [22]:
''' write down codes here '''
#conv_with_dilation_rate2 = ...
#conv_with_dilation_rate4 = ...
#conv_with_dilation_rate8 = ...
#conv_with_dilation_rate16 = ...

In [23]:
feature_map = front(x)
feature_map_rate2 = conv_with_dilation_rate2(feature_map)
feature_map_rate4 = conv_with_dilation_rate4(feature_map)
feature_map_rate8 = conv_with_dilation_rate8(feature_map)
feature_map_rate16 = conv_with_dilation_rate16(feature_map)

In [24]:
sppp = tf.concat([feature_map_rate2, feature_map_rate4, feature_map_rate8, feature_map_rate16], axis=3)
sppp

<tf.Tensor 'concat_1:0' shape=(1, 14, 14, 2048) dtype=float32>

---

# Quiz 7 : Building toy Unet

![unet](simple_unet.png)

## Instruction
implement a simple U-Net module using `ds`, `upsample`, and `deconv` module

In [25]:
def UNet_layer3(x):
    
    ds = layers.MaxPooling2D((2, 2))
    upsample = layers.UpSampling2D((2, 2))
    deconv = layers.Conv2DTranspose(3, (3, 3), activation='relu', strides=2, padding='same')
    
    out = ds(x)
    out_up = upsample(out)
    out_large = deconv(out)
    out = tf.concat([out_up, out_large], axis=3)
    return out

In [26]:
UNet_layer3(x)

<tf.Tensor 'concat_2:0' shape=(1, 224, 224, 6) dtype=float32>